### Need one sketch per set of MAGs (from one metaG)
- Flat sketch, k21. Also do at k31

In [ ]:
# merge signatures. 
srun --account=ctbrowngrp -p med2 -J calc-pres -t 4:00:00 -c 24 --mem=80gb --pty bash

# run the script
mamba activate branchwater-skipmer

snakemake --use-conda --resources mem_mb=120000 --rerun-triggers mtime \
-c 120 --rerun-incomplete -k -s sketch_mag.smk -n

In [ ]:
# merge signatures. 
srun --account=ctbrowngrp -p med2 -J calc-pres -t 4:00:00 -c 24 --mem=80gb --pty bash

# run the script
mamba activate branchwater-skipmer

snakemake --use-conda --resources mem_mb=80000 --rerun-triggers mtime \
-c 24 --rerun-incomplete -k -s snakefiles/sketch_mag.smk -n

### Create species level databases at k31
- One for GTDB
- One for GTDB + MAGS. 
- Compare genomes or reps?

Run x metagenomes
What species are more in the metagenomes when adding new speices

DB with one sketch per species: What species gained the most kmers? 
Compare the f_unique_weighted sum between species? 
Or just the sketch size? Num kmers. 

(Does colton already have a species level db?)

- Run gather x species db 
- Tax classify the output. 

https://github.com/dib-lab/sourmash_plugin_pangenomics


In [ ]:
# create the pangenome db 
mamba activate pangenomics_dev
sourmash scripts pangenome_createdb \
/group/ctbrowngrp5/sourmash-db/gtdb-rs226/gtdb-rs226.k21.rocksdb \
-t /group/ctbrowngrp5/sourmash-db/gtdb-rs226/gtdb-rs226.lineages.csv
-k 31 --scaled 10000 -o gtdb-rs226.pangenomedb_species.10k.zip

In [ ]:
sourmash scripts pangenome_createdb \
    gtdb-rs214-agatha-k21.zip \
    -t gtdb-rs214-agatha.lineages.csv.gz \
    -o agatha-merged.sig.zip --abund -k 21

In [ ]:
# create db
sourmash scripts manysketch -p dna,k=21,k=31,k=51,scaled=1000,abund \
-c 24 manysketch_all.csv -o MAGs.all.zip

# then index them for a rocksdb 
sourmash index MAGs.all_k31.rocksdb -F rocksdb -k 31 MAGs.all.zip

# figure out the lineages file 

In [ ]:
## Now do taxonomy for all the gather results. 
# create a from file where the gather results are.
find ./gather -type f -name "*.gtdb.csv" > fp.gather_out_gtdb.txt
find ./gather -type f -name "*.mags_and_gtdb.csv" > fp.gather_out_mags_and_gtdb.txt

# merge signatures. 
srun --account=ctbrowngrp -p med2 -J calc-pres -t 4:00:00 -c 24 --mem=80gb --pty bash

# run the script
mamba activate branchwater-skipmer

# tax
sourmash tax annotate \
--taxonomy ../gtdb_pangenomedb/gtdb-rs226.lineages.csv \
--from-file fp.gather_out_gtd1.txt \
--output-dir ./tax

sourmash tax metagenome \
--taxonomy ../gtdb_pangenomedb/gtdb-rs226.lineages.csv \
--from-file fp.gather_out_gtdb.txt -F csv_summary \
--output-dir ./tax -o gtdb_pig.tax

sourmash tax metagenome \
--taxonomy ../gtdb_pangenomedb/250807_gtdb_sraMAGs.lineages.db \
--from-file fp.gather_out_mags_and_gtdb.txt -F csv_summary \
--output-dir ./tax -o gtdb_sraMAGs_pig.tax


In [ ]:
../results/gtdb_pangenomedb/gtdb-rs226.lineages.csv
../results/gtdb_pangenomedb/250807_gtdb_sraMAGs.lineages.db

sourmash tax metagenome
    --gather-csv HSMA33MX_gather_x_gtdbrs202_k31.csv \
    --taxonomy gtdb-rs202.taxonomy.v2.csv

sourmash tax metagenome --gather-csv \
../gather/ERR1135182.gtdb.csv --taxonomy ../../gtdb_pangenomedb/gtdb-rs226.lineages.csv \
-o ERR1135182.gtdb

sourmash tax metagenome --gather-csv \
../gather/ERR1135182.mags_and_gtdb.csv --taxonomy ../../gtdb_pangenomedb/250807_gtdb_sraMAGs.lineages.db \
-o ERR1135182.mags_and_gtdb

# will prons ne
